In [28]:
import sys
!{sys.executable} -m pip install holidays

In [29]:
import holidays
print("✅ holidays geladen aus:", holidays.__file__)

✅ holidays geladen aus: c:\Daten\Weiterbildung\Python\Portfolioprojekt\Test_Projekt\.venv\Lib\site-packages\holidays\__init__.py


In [ ]:
# data_prep.ipynb
# --------------------------
# Notebook: DE Load + Price + Full Weather
# Purpose: Download, clean, join, feature-engineer hourly 60-min resolution dataset


# Cell 1: Imports & env setup
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta
import holidays
import requests
import os




In [30]:


raw_path = Path("../data/raw")
raw_path.mkdir(parents=True, exist_ok=True)

# Beispiel-Koordinate: Deutschland (Berlin)
latitude  = 52.52
longitude = 13.405
start_date = "2015-01-01"
end_date   = "2025-10-29"

url = (
    f"https://archive-api.open-meteo.com/v1/archive?"
    f"latitude={latitude}&longitude={longitude}"
    f"&start_date={start_date}&end_date={end_date}"
    f"&hourly=temperature_2m,wind_speed_10m,cloudcover"
    f"&timezone=Europe/Berlin"
)

print("🔍 Anfrage URL:", url)
r = requests.get(url)
r.raise_for_status()
data = r.json()

df = pd.DataFrame(data["hourly"])
df["time"] = pd.to_datetime(df["time"])
df = df.set_index("time")

# Umbenennen
df = df.rename(columns={
    "temperature_2m": "temp_C",
    "wind_speed_10m": "wind_m_s",
    "cloudcover": "cloud_cover_pct"
})

# Optional resample auf 60min (wenn nötig)
df = df.resample("60min").mean()

# Speichern
out_file = raw_path / "open_meteo_berlin_hourly.csv"
df.to_csv(out_file)
print("✅ Wetterdaten gespeichert unter:", out_file)
print(df.head())

🔍 Anfrage URL: https://archive-api.open-meteo.com/v1/archive?latitude=52.52&longitude=13.405&start_date=2015-01-01&end_date=2025-10-29&hourly=temperature_2m,wind_speed_10m,cloudcover&timezone=Europe/Berlin
✅ Wetterdaten gespeichert unter: ..\data\raw\open_meteo_berlin_hourly.csv
                     temp_C  wind_m_s  cloud_cover_pct
time                                                  
2015-01-01 00:00:00     4.0      14.2             87.0
2015-01-01 01:00:00     3.8      14.4             71.0
2015-01-01 02:00:00     3.6      14.9             69.0
2015-01-01 03:00:00     3.3      14.6             95.0
2015-01-01 04:00:00     3.0      14.1             82.0


In [23]:
# Cell 2: Define paths
DATA_DIR = Path('../data/raw')
OPS_DATA_CSV = DATA_DIR / 'time_series_60min_singleindex.csv'
DWD_WEATHER_CSV = DATA_DIR / 'dwd_hourly_weather.csv'
OUTPUT_DIR = Path('./processed')
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)


In [31]:
# Cell 3: Load OPSD data
# opsd_df = pd.read_csv(OPS_DATA_CSV, parse_dates=['utc_timestamp'], index_col='utc_timestamp')
opsd_df = pd.read_csv(
    OPS_DATA_CSV,
    sep=",",
    parse_dates=["utc_timestamp"],
    index_col="utc_timestamp",
)

print("Verfügbare DE-Spalten (Auszug):")
print([col for col in opsd_df.columns if col.startswith("DE")][:15], "...")

# === Relevante deutsche Spalten auswählen ===
opsd_df = opsd_df[
    [
        "DE_load_actual_entsoe_transparency",  # Stromverbrauch (real)
        "DE_LU_price_day_ahead",               # Day-Ahead-Preis (DE+LU)
        "DE_wind_generation_actual",           # Wind (gesamt)
        "DE_solar_generation_actual",          # Solar (gesamt)
    ]
].rename(
    columns={
        "DE_load_actual_entsoe_transparency": "load",
        "DE_LU_price_day_ahead": "price",
        "DE_wind_generation_actual": "wind",
        "DE_solar_generation_actual": "solar",
    }
)
# Auf Stundenebene sicherstellen (falls Lücken)
opsd_df = opsd_df.resample("60min").mean()

# Fehlende Werte auffüllen
opsd_df = opsd_df.interpolate(limit_direction="both")

# Vorschau
display(opsd_df.head())
print(f"\nZeitbereich: {opsd_df.index.min()} → {opsd_df.index.max()}")
print(f"Anzahl Zeilen: {len(opsd_df):,}")




Verfügbare DE-Spalten (Auszug):
['DE_load_actual_entsoe_transparency', 'DE_load_forecast_entsoe_transparency', 'DE_solar_capacity', 'DE_solar_generation_actual', 'DE_solar_profile', 'DE_wind_capacity', 'DE_wind_generation_actual', 'DE_wind_profile', 'DE_wind_offshore_capacity', 'DE_wind_offshore_generation_actual', 'DE_wind_offshore_profile', 'DE_wind_onshore_capacity', 'DE_wind_onshore_generation_actual', 'DE_wind_onshore_profile', 'DE_50hertz_load_actual_entsoe_transparency'] ...


,load,price,wind,solar
utc_timestamp,,,,
2014-12-31 23:00:00+00:00,41151.0,56.1,8852.0,71.0
2015-01-01 00:00:00+00:00,41151.0,56.1,8852.0,71.0
2015-01-01 01:00:00+00:00,40135.0,56.1,9054.0,71.0
2015-01-01 02:00:00+00:00,39106.0,56.1,9070.0,71.0
2015-01-01 03:00:00+00:00,38765.0,56.1,9163.0,71.0



Zeitbereich: 2014-12-31 23:00:00+00:00 → 2020-09-30 23:00:00+00:00
Anzahl Zeilen: 50,401


In [ ]:
# import os
# print(os.getcwd())

c:\Daten\Weiterbildung\Python\Portfolioprojekt\Test_Projekt\notebooks


In [32]:
import pandas as pd
import requests
import io
import zipfile
from pathlib import Path

# ---------------------------------------------
# 1️⃣ Einstellungen
# ---------------------------------------------
BASE_URL = "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly"
CATEGORIES = {
    "TU": "Lufttemperatur",
    "FF": "Windgeschwindigkeit",
    "SD": "Sonnenscheindauer"
}

STATIONS = {
    "NORTH": "10147",   # Hamburg
    "SOUTH": "10865",   # München
    "EAST": "10488",    # Dresden
    "WEST": "10400",    # Düsseldorf
    "CENTER": "10637"   # Erfurt
}

SAVE_PATH = Path("data/raw/weather_germany")
SAVE_PATH.mkdir(parents=True, exist_ok=True)

# ---------------------------------------------
# 2️⃣ Hilfsfunktion: URL prüfen
# ---------------------------------------------
def url_exists(url: str) -> bool:
    resp = requests.head(url)
    return resp.status_code == 200

# ---------------------------------------------
# 3️⃣ Downloadfunktion (sucht automatisch passende Datei)
# ---------------------------------------------
def download_dwd_file(cat: str, station: str):
    # Neuere Struktur seit 2024:
    # /.../hourly/{cat}/historical/ oder /recent/
    base_paths = [f"{BASE_URL}/{cat}/recent/", f"{BASE_URL}/{cat}/historical/"]
    file_patterns = [
        f"stundenwerte_{cat}_{station}_akt.zip",
        f"stundenwerte_{cat}_{station}_hist.zip",
        f"stundenwerte_{cat}_{station}.zip"
    ]

    for base in base_paths:
        for filename in file_patterns:
            url = base + filename
            if url_exists(url):
                print(f"⬇️ Lade {cat} von {url}")
                r = requests.get(url)
                z = zipfile.ZipFile(io.BytesIO(r.content))
                z.extractall(SAVE_PATH / f"{cat}_{station}")
                return True
    print(f"⚠️ Keine Datei für {cat} {station} gefunden.")
    return False

# ---------------------------------------------
# 4️⃣ Alles herunterladen
# ---------------------------------------------
all_dfs = []
for region, station in STATIONS.items():
    print(f"\n📍 Lade Daten für {region} (Station {station})")
    region_data = {}

    for cat in CATEGORIES.keys():
        if download_dwd_file(cat, station):
            # Entpackte Datei suchen
            extracted_files = list((SAVE_PATH / f"{cat}_{station}").glob("produkt_*.txt"))
            if not extracted_files:
                continue
            df = pd.read_csv(
                extracted_files[0], sep=";", na_values="-999",
                parse_dates=["MESS_DATUM"], index_col="MESS_DATUM"
            )
            region_data[cat] = df

    if region_data:
        # Kombinieren und umbenennen
        combined = pd.DataFrame(index=region_data["TU"].index)
        combined["temp"] = region



📍 Lade Daten für NORTH (Station 10147)
⚠️ Keine Datei für TU 10147 gefunden.
⚠️ Keine Datei für FF 10147 gefunden.
⚠️ Keine Datei für SD 10147 gefunden.

📍 Lade Daten für SOUTH (Station 10865)
⚠️ Keine Datei für TU 10865 gefunden.
⚠️ Keine Datei für FF 10865 gefunden.
⚠️ Keine Datei für SD 10865 gefunden.

📍 Lade Daten für EAST (Station 10488)
⚠️ Keine Datei für TU 10488 gefunden.
⚠️ Keine Datei für FF 10488 gefunden.
⚠️ Keine Datei für SD 10488 gefunden.

📍 Lade Daten für WEST (Station 10400)
⚠️ Keine Datei für TU 10400 gefunden.
⚠️ Keine Datei für FF 10400 gefunden.
⚠️ Keine Datei für SD 10400 gefunden.

📍 Lade Daten für CENTER (Station 10637)
⚠️ Keine Datei für TU 10637 gefunden.
⚠️ Keine Datei für FF 10637 gefunden.
⚠️ Keine Datei für SD 10637 gefunden.


In [33]:
import pandas as pd
from pathlib import Path

# ---------------------------------------------
# 1️⃣ Pfade definieren
# ---------------------------------------------
weather_path = Path("data/weather_germany_avg_2015_2020.csv")
opsd_path = Path("data/time_series_60min_singleindex.csv")

# ---------------------------------------------
# 2️⃣ Laden der Daten
# ---------------------------------------------
print("📂 Lade Wetterdaten ...")
weather_df = pd.read_csv(weather_path, parse_dates=["timestamp"], index_col="timestamp")

print("📂 Lade OPSD-Daten ...")
opsd_df = pd.read_csv(opsd_path, parse_dates=["utc_timestamp"], index_col="utc_timestamp")

# ---------------------------------------------
# 3️⃣ Zeitraum anpassen
# ---------------------------------------------
start, end = weather_df.index.min(), weather_df.index.max()
opsd_df = opsd_df.loc[start:end]

# ---------------------------------------------
# 4️⃣ Relevante Spalten für Deutschland auswählen
# ---------------------------------------------
columns = [
    "DE_load_actual_entsoe_transparency",
    "DE_LU_price_day_ahead",
    "DE_solar_generation_actual",
    "DE_wind_generation_actual",
    "DE_wind_onshore_generation_actual",
    "DE_wind_offshore_generation_actual"
]

# Nur vorhandene Spalten übernehmen
opsd_selected = opsd_df[[col for col in columns if col in opsd_df.columns]].copy()

# ---------------------------------------------
# 5️⃣ Zusammenführen mit Wetterdaten
# ---------------------------------------------
merged_df = pd.merge(
    opsd_selected, weather_df,
    how="inner", left_index=True, right_index=True
)

# ---------------------------------------------
# 6️⃣ Fehlende Werte auffüllen
# ---------------------------------------------
merged_df = merged_df.interpolate(limit_direction="both")

# ---------------------------------------------
# 7️⃣ Speichern
# ---------------------------------------------
output_path = Path("data/opsd_weather_merged_2015_2020.csv")
merged_df.to_csv(output_path)

print(f"\n✅ Kombinierte Datei gespeichert unter:\n{output_path}")
print(f"🧭 Zeitraum: {merged_df.index.min()} → {merged_df.index.max()}")
print(f"📈 Spalten: {list(merged_df.columns)}")
print("\n📊 Vorschau:")
print(merged_df.head())


📂 Lade Wetterdaten ...


FileNotFoundError: [Errno 2] No such file or directory: 'data\\weather_germany_avg_2015_2020.csv'

In [ ]:
# Cell 4: Load DWD weather data
weather_df = pd.read_csv(DWD_WEATHER_CSV, parse_dates=['utc_timestamp'], index_col='utc_timestamp')
# Assume weather_df has columns: temp, wind_speed, radiation, cloud_cover
weather_df = weather_df.resample('60min').mean()


In [ ]:
# Cell 5: Merge datasets
full_df = opsd_df.join(weather_df, how='left')


In [ ]:
# Cell 6: Missing value handling
full_df = full_df.ffill(limit=2) # forward fill up to 2 hours
full_df['missing_weather_flag'] = full_df[['temp', 'wind_speed', 'radiation', 'cloud_cover']].isna().any(axis=1)


In [ ]:
# Cell 7: Feature engineering - datetime features
full_df['hour'] = full_df.index.hour
full_df['day_of_week'] = full_df.index.dayofweek
full_df['is_weekend'] = full_df['day_of_week'].isin([5,6]).astype(int)
full_df['day_of_year'] = full_df.index.dayofyear
# seasonal encoding
full_df['hour_sin'] = np.sin(2*np.pi*full_df['hour']/24)
full_df['hour_cos'] = np.cos(2*np.pi*full_df['hour']/24)
full_df['dow_sin'] = np.sin(2*np.pi*full_df['day_of_week']/7)
full_df['dow_cos'] = np.cos(2*np.pi*full_df['day_of_week']/7)


In [ ]:
# Holidays
de_holidays = holidays.Germany()
full_df['is_holiday'] = full_df.index.to_series().apply(lambda x: 1 if x in de_holidays else 0)


In [ ]:
# Cell 8: Lag features (load and price)
lags = [1,2,3,24,48,168]
for lag in lags:
full_df[f'DE_load_lag{lag}'] = full_df['DE_load_actual'].shift(lag)
full_df[f'DE_price_lag{lag}'] = full_df['DE_price_day_ahead'].shift(lag)



In [ ]:
# Cell 9: Rolling statistics
windows = [3,24,168]
for w in windows:
full_df[f'DE_load_rollmean{w}'] = full_df['DE_load_actual'].rolling(w).mean()
full_df[f'DE_load_rollstd{w}'] = full_df['DE_load_actual'].rolling(w).std()
full_df[f'DE_price_rollmean{w}'] = full_df['DE_price_day_ahead'].rolling(w).mean()
full_df[f'DE_price_rollstd{w}'] = full_df['DE_price_day_ahead'].rolling(w).std()


In [ ]:

# Cell 10: Save train/val/test splits
train_df = full_df[:'2022-12-31']
val_df = full_df['2023-01-01':'2023-12-31']
test_df = full_df['2024-01-01':]

train_df.to_parquet(OUTPUT_DIR / 'train.parquet')
val_df.to_parquet(OUTPUT_DIR / 'val.parquet')
test_df.to_parquet(OUTPUT_DIR / 'test.parquet')


In [ ]:
# Cell 11: Quick sanity check
print(train_df.shape, val_df.shape, test_df.shape)
print(train_df.head())